In [1]:
import os
import pickle
import tarfile
import numpy as np

def extract_cifar10(file_path, extract_path="./cifar-10-batches-py"):
    if not os.path.exists(extract_path):
        with tarfile.open(file_path, "r:gz") as tar:
            tar.extractall()
    return extract_path

def load_batch(batch_file):
    with open(batch_file, 'rb') as f:
        entry = pickle.load(f, encoding='latin1')
        data = entry['data']
        labels = entry['labels']
        data = data.reshape(-1, 3, 32, 32).astype(np.uint8)
        data = np.transpose(data, (0, 2, 3, 1))  # (N, H, W, C)
        return data, labels


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CIFAR10Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data  # keep as numpy array!
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)  # Transform will convert it to tensor
        return img, label


def prepare_dataloaders(data_path, batch_size=128):
    # Load training data
    train_data, train_labels = [], []
    for i in range(1, 6):
        data, labels = load_batch(os.path.join(data_path, f"data_batch_{i}"))
        train_data.append(data)
        train_labels += labels
    train_data = np.concatenate(train_data)

    # Load test data
    test_data, test_labels = load_batch(os.path.join(data_path, "test_batch"))

    # Transform (PyTorch tensors, normalization optional)
    transform = transforms.Compose([
        transforms.ToTensor(),  # Already 0-1, so ToTensor suffices
    ])

    train_set = CIFAR10Dataset(train_data, train_labels, transform)
    test_set = CIFAR10Dataset(test_data, test_labels, transform)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


In [3]:
import torch.nn as nn
import torch.optim as optim

class SimpleVGG(nn.Module):
    def __init__(self):
        super(SimpleVGG, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

def train_model(model, train_loader, test_loader, device, epochs=50):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")
        test_model(model, test_loader, device)

def test_model(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, preds = torch.max(outputs, 1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)
    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
data_dir = extract_cifar10("cifar-10-python.tar.gz")
train_loader, test_loader = prepare_dataloaders(data_dir)

# Model Training
model = SimpleVGG()
train_model(model, train_loader, test_loader, device)


Epoch 1, Loss: 610.9012
Test Accuracy: 59.08%
Epoch 2, Loss: 409.8334
Test Accuracy: 63.60%
Epoch 3, Loss: 329.3827
Test Accuracy: 67.84%
Epoch 4, Loss: 277.9752
Test Accuracy: 74.50%
Epoch 5, Loss: 242.6155
Test Accuracy: 71.25%
Epoch 6, Loss: 210.2327
Test Accuracy: 79.69%
Epoch 7, Loss: 183.7327
Test Accuracy: 80.56%
Epoch 8, Loss: 158.5323
Test Accuracy: 79.80%
Epoch 9, Loss: 135.3394
Test Accuracy: 83.71%
Epoch 10, Loss: 118.8376
Test Accuracy: 83.75%
Epoch 11, Loss: 103.2197
Test Accuracy: 85.21%
Epoch 12, Loss: 88.6320
Test Accuracy: 85.47%
Epoch 13, Loss: 74.5865
Test Accuracy: 85.50%
Epoch 14, Loss: 63.7061
Test Accuracy: 84.03%
Epoch 15, Loss: 55.2612
Test Accuracy: 84.08%
Epoch 16, Loss: 50.2034
Test Accuracy: 83.51%
Epoch 17, Loss: 42.2437
Test Accuracy: 83.02%
Epoch 18, Loss: 37.3087
Test Accuracy: 84.75%
Epoch 19, Loss: 33.8840
Test Accuracy: 85.34%
Epoch 20, Loss: 30.2838
Test Accuracy: 85.93%
Epoch 21, Loss: 27.7867
Test Accuracy: 83.80%
Epoch 22, Loss: 26.6447
Test Acc

KeyboardInterrupt: 